### Download Necessary Files

In [2]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [25]:
import warnings
warnings.filterwarnings('ignore')

### Excercise 11.1

#### Variables for Date of Birth Prediction

In [39]:
# get Pregency data
import nsfg

#Get the sucessfull Preg
preg = nsfg.ReadFemPreg()
live = preg[preg.prglngth>30]

In [40]:
# Top 10 Rsquare value

import pandas as pd
import statsmodels.api as sm

df = live
target = "prglngth"
predictors = [col for col in df.columns if col != target]  # Exclude target variable
results = []

for predictor in predictors:
    try:
        df_clean = df[[predictor, target]].dropna() # Drop NaN values
        if df_clean[predictor].nunique() < 2:  # Skip if there's not enough variation
                continue  
        x = sm.add_constant(df_clean[predictor])  # Add intercept
        y = df_clean[target]
        model = sm.OLS(y, x).fit()
        r2 = model.rsquared
        results.append((predictor, r2))
    except Exception as e:
        print(f"Skipping {predictor} due to error: {e}")
   
# Convert to DataFrame and sort by R-squared
r2_df = pd.DataFrame(results, columns=["Variable", "R-squared"])
r2_df = r2_df.sort_values(by="R-squared", ascending=False).head(50)  

print (r2_df)  

         Variable  R-squared
4        moscurrp   1.000000
92      cmkidied3   1.000000
18        wksgest   0.810187
79      parenend2   0.679366
14       ageatend   0.453650
91      alivenow3   0.375000
51        weeksdk   0.304764
112    whatmeth04   0.257449
28      lobthwgt2   0.250000
94       lastage3   0.250000
74      cmkidied2   0.212656
46        pnctrim   0.209709
8        multbrth   0.175005
9        cmotpreg   0.156342
81      fedsolid2   0.129106
209   totalwgt_lb   0.124457
22    birthwgt_lb   0.119773
134     whynouse3   0.110054
149          lbw1   0.103725
90      livehere3   0.100000
19        mosgest   0.098279
83    frsteatd_p2   0.071683
78      legagree2   0.062500
2       howpreg_n   0.058602
89    matchfound3   0.055556
30   birthwgt_lb3   0.055328
77      wherenow2   0.054483
71    matchfound2   0.051562
31   birthwgt_oz3   0.040943
6        pregend2   0.034786
26   birthwgt_lb2   0.027763
111    whatmeth03   0.026488
87    ageqtnur_p2   0.022520
177    prglngt

Based on the above R-Squared value. I believe following can be right varaiable for predicting birth date

wksgest
weeksdk

In [55]:
import pandas as pd
import statsmodels.api as sm

predictors = ['wksgest','weeksdk']

# Keep only columns present in the dataset
predictors = [col for col in predictors if col in df.columns]

# Drop missing values for selected columns
df_clean = df.dropna(subset=[target] + predictors)
    
# Define independent (X) and dependent (y) variables
X = df_clean[predictors]
y = df_clean[target]

# Add constant for the intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     1.130
Date:                Sat, 08 Feb 2025   Prob (F-statistic):              0.394
Time:                        17:38:57   Log-Likelihood:                -16.183
No. Observations:                   8   AIC:                             38.37
Df Residuals:                       5   BIC:                             38.60
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.5093      2.105     17.346      0.000      31.099      41.920
wksgest       -0.0098      0.045     -0.218      0.836      -0.125       0.106
weeksdk        0.4133      0.283      1.460      0.204      -0.314       1.141
==============================================================================
Omnibus:                        0.987   Durbin-Watson:                   1.734
Prob(Omnibus):                  0.611   Jarque-Bera (JB):                0.595
Skew:                          -0.144   Prob(JB):                        0.743
Kurtosis:                       1.696   Cond. No.                         128.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Excercise 11.3

#### No of Child Prediction

In [69]:
import pandas as pd
import statsmodels.api as sm
import nsfg

# Join preg and resp data
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

# Select relevant predictors and drop missing values
df = join[['numbabes', 'age_r', 'race', 'educat', 'totincr']].dropna()

# Define dependent (target) and independent (predictor) variables
X = df.drop(columns=['numbabes'])  # Predictors
y = df['numbabes']  # Target (count variable)

# Add constant for intercept
X = sm.add_constant(X)

# Fit Poisson regression model
poisson_model = sm.Poisson(y, X).fit()

# Print model summary
print(poisson_model.summary())

Optimization terminated successfully.
         Current function value: 1.689698
         Iterations 5
                          Poisson Regression Results                          
Dep. Variable:               numbabes   No. Observations:                 9011
Model:                        Poisson   Df Residuals:                     9006
Method:                           MLE   Df Model:                            4
Date:                Sat, 08 Feb 2025   Pseudo R-squ.:                 0.03051
Time:                        19:39:37   Log-Likelihood:                -15226.
converged:                       True   LL-Null:                       -15705.
Covariance Type:            nonrobust   LLR p-value:                3.583e-206
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1345      0.049     23.370      0.000       1.039       1.230
age_r          0.0211      0.

#### Predicting no of childrens

In [71]:
# Create a DataFrame for the specific person
person = pd.DataFrame({
    'const': [1],        # Intercept
    'age_r': [35],         
    'race': [1],   
    'educat': [16],  #16 years foe graduate
    'totincr': [14]  # Code 14 if the income is 75k or more
})

# Ensure all other categorical dummy variables are set to 0
for col in X.columns:
    if col not in person.columns:
        person[col] = 0

# Predict expected number of children
predicted_children = poisson_model.predict(person)
print(f"Predicted number of children: {predicted_children[0]:.2f}")

Predicted number of children: 2.23


### Excercise 11.4

#### Create a Model

In [78]:
import pandas as pd
import statsmodels.api as sm

# Select relevant predictors and drop missing values
df = join[['rmarital', 'age_r', 'race', 'educat', 'totincr']].dropna()

# Define dependent (target) and independent (predictor) variables
X = df.drop(columns=['rmarital'])  # Predictors
y = df['rmarital']  # Target (categorical variable)

# Add constant for intercept
X = sm.add_constant(X)

# Fit multinomial logistic regression model
mnlogit_model = sm.MNLogit(y, X).fit()

# Print model summary
print(mnlogit_model.summary())

Optimization terminated successfully.
         Current function value: 1.100928
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:               rmarital   No. Observations:                 9011
Model:                        MNLogit   Df Residuals:                     8986
Method:                           MLE   Df Model:                           20
Date:                Sat, 08 Feb 2025   Pseudo R-squ.:                  0.1563
Time:                        19:59:44   Log-Likelihood:                -9920.5
converged:                       True   LL-Null:                       -11758.
Covariance Type:            nonrobust   LLR p-value:                     0.000
rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7360      0.302     15.658      0.000       4.143       5.329
age_r         -0.0571      0.

#### Getting Married Prediction

In [82]:
# Create a DataFrame for the specific person
person = pd.DataFrame({
    'const': [1],        
    'age_r': [25],         
    'race': [2],   # 2 for White
    'educat': [12], # 12 year for highschool
    'totincr': [11]  # 11 code for 45k income
})

# Ensure all other categorical dummy variables are set to 0
for col in X.columns:
    if col not in person.columns:
        person[col] = 0

# Predict probabilities for each marital status category
predicted_probs = mnlogit_model.predict(person)
print(predicted_probs.T)

          0
0  0.710814
1  0.140136
2  0.001094
3  0.033169
4  0.027406
5  0.087381
